In [1]:
import os

In [2]:
%pwd

'/Users/anishmitagar/Documents/repos/summer_2023_projects/end-to-end-diabetes-project/research'

In [4]:
os.chdir('/Users/anishmitagar/Documents/repos/summer_2023_projects/end-to-end-diabetes-project')

In [5]:
%pwd

'/Users/anishmitagar/Documents/repos/summer_2023_projects/end-to-end-diabetes-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    save_path: Path
    preprocessor_path: Path

In [7]:
from mlClassifier.constants import *
from mlClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            save_path=config.save_path,
            preprocessor_path=config.preprocessor_path
        )

        return data_transformation_config

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [10]:
import os
import sys
import dill
import pickle

def save_object(file_path, obj):

    dir_path = os.path.dirname(file_path)

    os.makedirs(dir_path, exist_ok=True)

    with open(file_path, "wb") as file_obj:
        dill.dump(obj, file_obj)
    

    
def load_object(file_path):

    with open(file_path, "rb") as file_obj:
        return pickle.load(file_obj)

In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def transform_and_save(self):
        df = pd.read_csv(self.config.data_path)
        df = df.drop_duplicates()

        X, y = df.drop(columns=['diabetes'],axis=1), pd.DataFrame(df['diabetes'])

        numerical_columns = list(X.select_dtypes(exclude="object").columns)
        categorical_columns = list(X.select_dtypes(include="object").columns)

        num_pipeline = Pipeline (
            steps=[
                ("scaler", MinMaxScaler())        
            ]
        )
        cat_pipeline = Pipeline (
            steps=[
                ("one_hot_encoder", OneHotEncoder()),
            ]
        )

        preprocessor = ColumnTransformer (
            [
                ("num_pipline", num_pipeline, numerical_columns),
                ("cat_pipline", cat_pipeline, categorical_columns)
            ]
        )

        X = pd.DataFrame(preprocessor.fit_transform(X))
        y.reset_index(drop=True, inplace=True)
        df = pd.concat([X, y], axis=1)
        df.to_csv(self.config.save_path, index=False)

        save_object(file_path=self.config.preprocessor_path, obj=preprocessor)


In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_and_save()
except Exception as e:
    raise e

[2023-07-22 10:25:14,930: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-22 10:25:14,935: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-22 10:25:14,935: INFO: common: created directory at: artifacts]
[2023-07-22 10:25:14,936: INFO: common: created directory at: artifacts/data_transformation]
